In [ ]:
!pip install -q ../input/kaggle-efficientnet-repo/efficientnet-1.0.0-py3-none-any.whl

In [ ]:
!pip install tensorflow_addons

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import argparse
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from tensorflow.keras import layers as L
import efficientnet.tfkeras as efn
import tensorflow.keras.backend as K

In [ ]:
import tensorflow_addons as tfa

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
train_fns = ['gs://kds-fdd26a53c479ba2fe0fd09aefd6ba5b5da014d103663a027f2ba8da1/0train_train00-100000.tfrec',
 'gs://kds-4a274b96dcced2a2d564739ab3265794bbd458d71e6eb5e99ff6a578/0train_train00-100000.tfrec',
 'gs://kds-839ccc6d5a39024648dbb162ab8c72a9ee59169297f57a257abde0c7/0train_train00-100000.tfrec',
 'gs://kds-514346ad1ef1306c46b1722bdf0cac13e3b4138d9302eb4b1b820f4f/0train_train00-100000.tfrec',
 'gs://kds-f6b193f82266500b631303010d4faaffc0d6a18593870dc1b948ef89/0train_train00-100000.tfrec',
 'gs://kds-094c51c80e99a45ee7b4d9bf3a6bd50a3766e08cdecea2cf34a478a4/0train_train00-100000.tfrec',
 'gs://kds-7348669340cc33cba51450cdf383dd082456377884297dfe1d1b6328/0train_train00-100000.tfrec',
 'gs://kds-8b955b9069a2f230c0f031da2a3f7dd122dcb542e2e2f583e400b057/0train_train00-100000.tfrec',
 'gs://kds-9c24f19b758d727268d0efab4dde21d5073c06828c3e3c7d8c346307/0train_train00-100000.tfrec',
 'gs://kds-8fcb00abe16292f58c5dc34a3c24dace10e2004ec5848e046d13a2c8/0train_train00-100000.tfrec',
 'gs://kds-8b562794f0c7cf738927f8df0abb12d6ad4756dc9cab0d0b0f599d47/0train_train00-100000.tfrec',
 'gs://kds-175a817f8c2bd0b9371a16f8d89f410f1b276a30cd1ad91d80c0c31b/0train_train00-100000.tfrec',
 'gs://kds-67c4a21d71f589012a90c7555f59f64551f1080920a1ac15a4bbc304/0train_train00-100000.tfrec',
 'gs://kds-d642de0737aa752538cfdbbde6538f25efd787349d20696fbb06229b/0train_train00-100000.tfrec',
 'gs://kds-0da7840c65cf60c5c69365e869ec091af5eb8c34cf18611639c2f661/0train_train00-175912.tfrec']

val_fns = ['gs://kds-e6a7a5fbb73e871113c2a6e6506a5d29ce98c7fe70295e8f9bb3f1c8/0val_train00-4558.tfrec']

In [ ]:
def get_strategy():
  # Detect hardware, return appropriate distribution strategy
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
        print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        strategy = tf.distribute.get_strategy()

    print('REPLICAS: ', strategy.num_replicas_in_sync)
    return strategy
strategy = get_strategy()

In [ ]:
backbone_name = 'efficientnet-b0'

In [ ]:
# class ConvNet(tf.keras.Model):

#     def __init__(self, engine, input_shape, weights):
#         super(ConvNet, self).__init__()
        
#         self.engine = engine(
#             include_top=False, input_shape=input_shape, weights=weights)
        
        
#         self.flat = tf.keras.layers.GlobalAveragePooling2D()
# #         self.flat = tf.keras.layers.Flatten()
#         self.dropout = tf.keras.layers.Dropout(0.3)
#         self.dropout1 = tf.keras.layers.Dropout(0.3)
#         self.dense_1 = tf.keras.layers.Dense(1024)

#     @tf.function
#     def call(self, inputs, **kwargs):
#         x = self.engine(inputs)
#         x = self.flat(x)
#         x = tf.math.l2_normalize(x, axis=1)
#         x = self.dropout(x, training=kwargs.get('training', False))
#         x = self.dense_1(x)
#         x = tf.math.l2_normalize(x, axis=1)
#         return x


In [ ]:
def build_model(engine, input_shape, weights):
    
    inp = tf.keras.layers.Input(shape=input_shape)
    base = engine(include_top=False, input_shape=input_shape, weights=weights)
    
    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024)(x)
    x = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    model = tf.keras.Model(inputs=[inp],outputs=[x])
    return model

In [ ]:
if backbone_name.startswith('efficientnet'):
    model_fn = getattr(efn, f'EfficientNetB{backbone_name[-1]}')

In [ ]:
IMG_SIZE = 256

In [ ]:
with strategy.scope():
    model = build_model(engine=model_fn, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet') 
    model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.001), loss=tfa.losses.TripletSemiHardLoss(margin=0.5))    

In [ ]:
model.summary()

In [ ]:
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min',
    min_delta=0.0001, cooldown=0, min_lr=0.000001)
checkpoint = tf.keras.callbacks.ModelCheckpoint(backbone_name+'.h5', monitor='val_loss',
    verbose=1, save_best_only=True, save_weights_only=True, mode='min')

In [ ]:
def flip(x: tf.Tensor) -> tf.Tensor:
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)
    return x

def decode_image(image_data):
#     image = tf.io.decode_jpeg(image_data, channels=3, dct_method='INTEGER_ACCURATE')
    image = tf.io.decode_jpeg(image_data, channels=3)
    image = tf.image.resize(image, [IMG_SIZE,IMG_SIZE])
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    return image



def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {}

    LABELED_TFREC_FORMAT['image'] = tf.io.FixedLenFeature([], tf.string)
    LABELED_TFREC_FORMAT['label'] = tf.io.FixedLenFeature([], tf.int64)
    
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    
    image = decode_image(example['image'])
    
    label = tf.cast(example['label'], tf.int32)
    return image, label


def load_dataset(filenames, labeled=True, ordered=False, training=True):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord)
    return dataset

def get_training_dataset(TRAINING_FILENAMES):
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(TEST_FILENAMES, ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=True, ordered=ordered, training=False)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

In [ ]:
num_train_samples = sum(int(fn.split('-')[-1].split('.')[0]) for fn in train_fns)
num_val_samples = sum(int(fn.split('-')[-1].split('.')[0]) for fn in val_fns)
num_train_samples, num_val_samples 

In [ ]:
BATCH_SIZE = 4096
STEPS_PER_EPOCH = num_train_samples // BATCH_SIZE
VAL_STEPS_PER_EPOCH = num_val_samples // BATCH_SIZE

In [ ]:
history = model.fit(get_training_dataset(train_fns), steps_per_epoch=STEPS_PER_EPOCH, verbose=1,
    validation_data = get_validation_dataset(val_fns), validation_steps=VAL_STEPS_PER_EPOCH, epochs=1, callbacks = [lr_callback, checkpoint])

In [ ]:
model.save_weights("model_iter_1.h5")